In [1]:
from datasets import load_dataset
import torch
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import time
from tqdm.notebook import tqdm
import numpy as np # linear algebra
import pandas as pd 

In [2]:
OUT_PATH="D://sweb//extracted//"
question_id = 2
timer=0

In [3]:
dataset = load_dataset("AmazonScience/mintaka",trust_remote_code=True)

In [4]:

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

data = dataset['test']

dataset2 = MyDataset(data)

df = pd.DataFrame([sample for sample in dataset2])
df

,id,lang,question,answerText,category,complexityType,questionEntity,answerEntity
0,fae46b21,en,What man was a famous American author and also...,Mark Twain,history,intersection,"[{'name': 'Q1497', 'entityType': 'entity', 'la...","[{'name': 'Q7245', 'label': 'Mark Twain'}]"
1,bc8713cc,en,How many Academy Awards has Jake Gyllenhaal be...,1,movies,count,"[{'name': 'Q133313', 'entityType': 'entity', '...","[{'name': 'Q106291', 'label': 'Academy Award f..."
2,d2a03f72,en,"Who is older, The Weeknd or Drake?",Drake,music,comparative,"[{'name': 'Q2121062', 'entityType': 'entity', ...","[{'name': 'Q33240', 'label': 'Drake'}]"
3,9a296167,en,How many children did Donald Trump have?,5,history,count,"[{'name': 'Q22686', 'entityType': 'entity', 'l...","[{'name': 'Q3713655', 'label': 'Donald Trump J..."
4,e343ad26,en,Is the main hero in Final Fantasy IX named Kuja?,No,videogames,yesno,"[{'name': 'Q474573', 'entityType': 'entity', '...",[]
...,...,...,...,...,...,...,...,...
3995,d52b03ee,en,Who was the first woman mayor of San Francisco?,Dianne Feinstein,politics,ordinal,"[{'name': 'Q62', 'entityType': 'entity', 'labe...","[{'name': 'Q230733', 'label': 'Dianne Feinstei..."
3996,07f2947c,en,Where was the last Republican mayor of Boston ...,"Portland, Maine",politics,ordinal,"[{'name': 'Q100', 'entityType': 'entity', 'lab...","[{'name': 'Q49201', 'label': 'Portland'}]"
3997,58727fb0,en,How long was the 53rd mayor of Boston in office?,"20 years, 6 months",politics,ordinal,"[{'name': 'Q100', 'entityType': 'entity', 'lab...",[]
3998,a25818a2,en,Who was the first British monarch to have a pr...,George I,politics,ordinal,"[{'name': 'Q145', 'entityType': 'entity', 'lab...","[{'name': 'Q130805', 'label': 'George I of Gre..."


In [5]:
def extract_question_entities(question_index):
    question_entities = []
    entity_list = df.iloc[question_index]["questionEntity"]
    for i in entity_list:
         question_entities.append(i["name"])
#     print(question_entities)
    return question_entities



In [6]:
def run_query(entities):
    entities[0]=" wd:"+entities[0]
    entities=" wd:".join(entities)
    
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)


    query = """
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wikibase: <http://wikiba.se/ontology#>
        
        SELECT ?subLabel ?p2Label ?valueLabel ?value
        WHERE {{
          VALUES ?entity {{ {0} }}
          ?entity ?property ?value.
          BIND(?entity as ?sub)

          ?p2 wikibase:directClaim ?property.   
          FILTER(STRSTARTS(STR(?property), "http://www.wikidata.org/prop/direct/"))
          SERVICE wikibase:label {{
            bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
          }}

    }}
     """.format(entities)


    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

   
    try:
        results = sparql.query().convert()
    except Exception as e:
         print("Error executing SPARQL query:", e)
         return None
    objects=[]
    triples=[]
    statements = results["results"]["bindings"]
    if statements:
        for statement in statements:
            object_id = statement["value"]["value"]
            property_value = statement["p2Label"]["value"]
            object_value = statement["valueLabel"]["value"]
            sub_value = statement["subLabel"]["value"]
           
            if object_id!=object_value:
                objects.append(str(object_id).split("/")[-1])
                triples.append(f"{sub_value} : {property_value} : {object_value}")
    
    return objects,triples



In [7]:
trips=[]
z=300
def hop_forward(entities,k):
    global timer
    if k==0:
        return
    if entities==[]:
        return
    all_objects=[]
    for i in range(0,len(entities),z):
        temp=entities[i:i+z]
        objects,triples = run_query(temp)
        all_objects.extend(objects)
        trips.extend(triples)
    hop_forward(all_objects,k-1)


In [8]:
def create_csv(all_triples,idx):
    triples=[]
    for i in all_triples:
        trip = i.split(" : ")
        triples.append(trip)
    df_triples = pd.DataFrame(triples,columns=["subject","predicate","object"])
    df_triples.to_csv(f"question_{idx}.csv")

In [9]:
def create_txt(all_triples,idx):
    # print(all_triples)
    with open(f"{OUT_PATH}question_{idx}.txt","w",encoding='utf-8') as f:
        for i in all_triples:
            trip = " ".join(i.split(" : "))
            f.write(trip)
            f.write("\n")
        f.close()
    # print("done")


In [10]:
def find_query(entity):
    
    endpoint_url = "https://query.wikidata.org/sparql"
    sparql = SPARQLWrapper(endpoint_url)


    query = """
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wikibase: <http://wikiba.se/ontology#>
        PREFIX wd: <http://www.wikidata.org/entity/>
        SELECT DISTINCT ?qid
        WHERE {{

        BIND( STRLANG("{0}", "en") AS ?label ) .
        ?item rdfs:label ?label .

        BIND(STRAFTER(STR(?item), STR(wd:)) AS ?qid) .

    }}
     """.format(entity)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

   
    try:
        results = sparql.query().convert()
    except Exception as e:
         print("Error executing SPARQL query:", e)
         return None
    
    statements = results["results"]["bindings"]
    return statements[0]['qid']['value']

In [11]:
def remove_occurences(l):
    l2=[]
    for i in range(len(l)):
        if l[i]!='None':
            if l[i][0]=="Q":
                l2.append(l[i])
            else:
                try:
                    x=find_query(l[i])
                    l2.append(x)
                except:
                    continue
                
    return l2

In [12]:
def ask_question(idx):
    global trips
    question_entities = list(set(extract_question_entities(idx)))
    # print(question_entities)
    question_entities=remove_occurences(question_entities)
    # print(question_entities)
    all_triples = []
    objects,triples = run_query(question_entities)
    remove_occurences(objects)
    all_triples.extend(triples)
    x=0
    hop_forward(objects,1)
    all_triples.extend(trips)
    trips=[]
    x+=1
            
    # create_csv(all_triples,idx)
    create_txt(all_triples,idx)
    return all_triples

In [15]:
# all_triples = ask_question(question_id)
# len(all_trples)4
#2565
#2721
#2722
start=800
end=1000
for i in tqdm(range(start,end)):
    ask_question(i)
    print(f"Question {i} : Done ")

  0%|          | 0/200 [00:00<?, ?it/s]

Question 800 : Done 
Question 801 : Done 
Question 802 : Done 
Question 803 : Done 
Question 804 : Done 
Question 805 : Done 
Question 806 : Done 
Question 807 : Done 
Question 808 : Done 
Question 809 : Done 
Question 810 : Done 
Question 811 : Done 
Question 812 : Done 
Question 813 : Done 
Question 814 : Done 
Question 815 : Done 
Question 816 : Done 
Question 817 : Done 
Question 818 : Done 
Question 819 : Done 
Question 820 : Done 
Question 821 : Done 
Question 822 : Done 
Question 823 : Done 
Question 824 : Done 
Question 825 : Done 
Question 826 : Done 
Question 827 : Done 
Question 828 : Done 
Question 829 : Done 
Question 830 : Done 
Question 831 : Done 
Question 832 : Done 
Question 833 : Done 
Question 834 : Done 
Question 835 : Done 
Question 836 : Done 
Question 837 : Done 
Question 838 : Done 
Question 839 : Done 
Question 840 : Done 
Question 841 : Done 
Question 842 : Done 
Question 843 : Done 
Question 844 : Done 
Question 845 : Done 
Question 846 : Done 
Question 847 